# Preprocessing for Weather and Price data


In [2]:
# Loading lib
import pandas as pd
from datetime import datetime

## Improt Price Data
the historical price data is extracted from [IESO](http://reports.ieso.ca/public/PriceHOEPPredispOR/)
- HOEP: Hourly Ontario Electricity Price
 (the target vataible)

In [3]:
url = 'http://reports.ieso.ca/public/PriceHOEPPredispOR/PUB_PriceHOEPPredispOR'
index_1 = '_2018.csv'
index_2 = '_2019.csv'
index_3 = '_2020.csv'
index_4 = '_2021.csv'
index_5 = '_2022.csv'

df1 = pd.read_csv(url+index_1, header=[3])
df2 = pd.read_csv(url+index_2, header=[3])
df3 = pd.read_csv(url+index_3, header=[3])
df4 = pd.read_csv(url+index_4, header=[3])
df5 = pd.read_csv(url+index_5, header=[3])

price = pd.concat([df1,df2,df3,df4,df5], axis=0)
price.reset_index(drop=True, inplace=True)
price.head()

,Date,Hour,HOEP,Hour 1 Predispatch,Hour 2 Predispatch,Hour 3 Predispatch,OR 10 Min Sync,OR 10 Min non-sync,OR 30 Min
0,2018-01-01,1,51.29,40.62,35.50,40.32,0.20,0.20,0.19
1,2018-01-01,2,43.59,40.93,40.57,30.22,0.20,0.20,0.20
2,2018-01-01,3,93.60,49.84,45.30,40.20,0.25,0.25,0.22
3,2018-01-01,4,54.78,55.60,40.04,45.30,0.20,0.20,0.20
4,2018-01-01,5,14.35,40.04,34.80,47.47,0.20,0.20,0.20


In [4]:
# select the HOEP, which is Hourly Ontario Electricity Price that needed
price['Date'] = pd.to_datetime(price['Date'].str.strip(), format='%Y/%m/%d')
price['Hour'] = price.Hour.astype('timedelta64[h]')
price['time']=price['Date']+price['Hour']

price=price[['time','HOEP']]
price

,time,HOEP
0,2018-01-01 01:00:00,51.29
1,2018-01-01 02:00:00,43.59
2,2018-01-01 03:00:00,93.60
3,2018-01-01 04:00:00,54.78
4,2018-01-01 05:00:00,14.35
...,...,...
39787,2022-07-16 20:00:00,69.31
39788,2022-07-16 21:00:00,67.07
39789,2022-07-16 22:00:00,65.55
39790,2022-07-16 23:00:00,64.23


In [5]:
# select the timeline between 2018-01-01 and 2022-06-20
split_date = pd.to_datetime('2022-06-21')
price = price.loc[price.time < split_date]
price

,time,HOEP
0,2018-01-01 01:00:00,51.29
1,2018-01-01 02:00:00,43.59
2,2018-01-01 03:00:00,93.60
3,2018-01-01 04:00:00,54.78
4,2018-01-01 05:00:00,14.35
...,...,...
39162,2022-06-20 19:00:00,64.71
39163,2022-06-20 20:00:00,64.20
39164,2022-06-20 21:00:00,40.74
39165,2022-06-20 22:00:00,29.66


## Import supply data
the historical supply data is extracted from [IESO](http://reports.ieso.ca/public/GenOutputbyFuelHourly/)
- hourly generated by fuel type and total output

In [7]:
import requests 
import re
import xml.etree.ElementTree as ET
import pandas as pd

url = 'http://reports.ieso.ca/public/GenOutputbyFuelHourly/PUB_GenOutputbyFuelHourly'
index_1 = '_2018.xml'
index_2 = '_2019.xml'
index_3 = '_2020.xml'
index_4 = '_2021.xml'
index_5 = '_2022.xml'
index=[index_1,index_2,index_3,index_4,index_5]


for m in index:
  r=requests.get(url+m)
  rr=re.sub("\<Document .*\>","<Document>",r.text)
  rr=re.sub("\<\?xml.* \?\>","",rr)
  xml_data=ET.fromstring(rr)
  l=[]

  for i in xml_data.findall(".//DailyData"):
      day=[i.find(".//Day").text]
      for j in i.findall(".//HourlyData"):
          Hour=[j.find(".//Hour").text]
          Fuel=[i.text for i in j.findall(".//Output")]
          total=sum(map(int,Fuel))
          output=day+Hour+Fuel+[total]
          l.append(output)

  df=pd.DataFrame(l, columns=['Date','Hour','NUCLEAR','GAS','HYDRO','WIND','SOLAR','BIOFUEL','Total'])
  if m == index[0]:
    temp = df
  elif m == index[1]:
    temp1 = df
  elif m == index[2]:
    temp2 = df
  elif m == index[3]:
    temp3 = df
  elif m == index[4]:
    temp4 = df

supply = pd.concat([temp,temp1,temp2,temp3, temp4], axis = 0)

supply 

,Date,Hour,NUCLEAR,GAS,HYDRO,WIND,SOLAR,BIOFUEL,Total
0,2018-01-01,1,11671,1583,4152,992,0,3,18401
1,2018-01-01,2,11672,1192,4093,1124,0,1,18082
2,2018-01-01,3,11672,1040,4231,1265,0,0,18208
3,2018-01-01,4,11669,1041,3895,1369,0,0,17974
4,2018-01-01,5,11674,1004,3271,1848,0,0,17797
...,...,...,...,...,...,...,...,...,...
4723,2022-07-16,20,10206,3993,4325,122,7,187,18840
4724,2022-07-16,21,10207,3674,4245,153,0,167,18446
4725,2022-07-16,22,10210,3198,3830,264,0,98,17600
4726,2022-07-16,23,10209,2202,3738,349,0,89,16587


Because real-time hourly fuel type supply demand is not available, we will use yesterday's data to forecast todays data. That means we are shifting the dataset by one day

In [8]:
import datetime
supply['Date'] = pd.to_datetime(supply['Date'].str.strip(), format='%Y/%m/%d')
supply['Date'] = supply['Date'] + datetime.timedelta(days=1) ## shifting the dataset by one date
supply['Hour'] = supply.Hour.astype('timedelta64[h]')
supply['time']=supply['Date']+supply['Hour']

supply=supply[['time','NUCLEAR','GAS','HYDRO','WIND','SOLAR','BIOFUEL','Total']]
supply

,time,NUCLEAR,GAS,HYDRO,WIND,SOLAR,BIOFUEL,Total
0,2018-01-02 01:00:00,11671,1583,4152,992,0,3,18401
1,2018-01-02 02:00:00,11672,1192,4093,1124,0,1,18082
2,2018-01-02 03:00:00,11672,1040,4231,1265,0,0,18208
3,2018-01-02 04:00:00,11669,1041,3895,1369,0,0,17974
4,2018-01-02 05:00:00,11674,1004,3271,1848,0,0,17797
...,...,...,...,...,...,...,...,...
4723,2022-07-17 20:00:00,10206,3993,4325,122,7,187,18840
4724,2022-07-17 21:00:00,10207,3674,4245,153,0,167,18446
4725,2022-07-17 22:00:00,10210,3198,3830,264,0,98,17600
4726,2022-07-17 23:00:00,10209,2202,3738,349,0,89,16587


In [9]:
# select the timeline between 2018-01-01 and 2022-06-20
split_date = pd.to_datetime('2022-06-21')
supply = supply.loc[supply.time < split_date]
supply

,time,NUCLEAR,GAS,HYDRO,WIND,SOLAR,BIOFUEL,Total
0,2018-01-02 01:00:00,11671,1583,4152,992,0,3,18401
1,2018-01-02 02:00:00,11672,1192,4093,1124,0,1,18082
2,2018-01-02 03:00:00,11672,1040,4231,1265,0,0,18208
3,2018-01-02 04:00:00,11669,1041,3895,1369,0,0,17974
4,2018-01-02 05:00:00,11674,1004,3271,1848,0,0,17797
...,...,...,...,...,...,...,...,...
4074,2022-06-20 19:00:00,9730,367,5185,358,33,0,15673
4075,2022-06-20 20:00:00,9728,384,5542,349,6,0,16009
4076,2022-06-20 21:00:00,9732,257,5674,344,1,1,16009
4077,2022-06-20 22:00:00,9729,214,5572,396,0,13,15924


## Import demand data
the historical demand data is extracted from [IESO](http://reports.ieso.ca/public/DemandZonal/)
- Ontario Demand is the target variable




In [11]:
url = 'http://reports.ieso.ca/public/DemandZonal/PUB_DemandZonal'
index_1 = '_2018.csv'
index_2 = '_2019.csv'
index_3 = '_2020.csv'
index_4 = '_2021.csv'
index_5 = '_2022.csv'

df1 = pd.read_csv(url+index_1, header=[3])
df2 = pd.read_csv(url+index_2, header=[3])
df3 = pd.read_csv(url+index_3, header=[3])
df4 = pd.read_csv(url+index_4, header=[3])
df5 = pd.read_csv(url+index_5, header=[3])

demand = pd.concat([df1,df2,df3,df4,df5], axis=0)
demand.reset_index(drop=True, inplace=True)
demand

,Date,Hour,Ontario Demand,Northwest,Northeast,Ottawa,East,Toronto,Essa,Bruce,Southwest,Niagara,West,Zone Total,Diff
0,2018-01-01,1,16627,591,1466,1066,1268,5340,1579,89,3157,477,1489,16522,-105
1,2018-01-01,2,16084,577,1420,985,1300,5211,1516,91,3061,462,1465,16086,2
2,2018-01-01,3,15866,613,1417,943,1316,5096,1471,86,3032,446,1441,15862,-4
3,2018-01-01,4,15725,656,1430,943,1303,4987,1451,81,2911,439,1413,15614,-112
4,2018-01-01,5,15470,657,1425,932,1343,4926,1422,69,2907,449,1391,15522,51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39787,2022-07-16,20,18696,535,1157,1209,1216,6887,1356,92,3674,681,1952,18759,63
39788,2022-07-16,21,18080,530,1145,1150,1171,6685,1300,89,3567,653,1877,18167,87
39789,2022-07-16,22,17088,528,1121,1102,1054,6401,1206,84,3324,608,1807,17233,145
39790,2022-07-16,23,15916,477,1172,1017,920,5956,1079,82,3101,559,1694,16058,142


In [12]:
demand['Date'] = pd.to_datetime(demand['Date'].str.strip(), format='%Y/%m/%d')
demand['Hour'] = demand.Hour.astype('timedelta64[h]')
demand['time']=demand['Date']+demand['Hour']

demand=demand[['time','Ontario Demand']]
demand

,time,Ontario Demand
0,2018-01-01 01:00:00,16627
1,2018-01-01 02:00:00,16084
2,2018-01-01 03:00:00,15866
3,2018-01-01 04:00:00,15725
4,2018-01-01 05:00:00,15470
...,...,...
39787,2022-07-16 20:00:00,18696
39788,2022-07-16 21:00:00,18080
39789,2022-07-16 22:00:00,17088
39790,2022-07-16 23:00:00,15916


In [13]:
# select the timeline between 2018-01-01 and 2022-06-20
split_date = pd.to_datetime('2022-06-21')
demand = demand.loc[demand.time < split_date]
demand

,time,Ontario Demand
0,2018-01-01 01:00:00,16627
1,2018-01-01 02:00:00,16084
2,2018-01-01 03:00:00,15866
3,2018-01-01 04:00:00,15725
4,2018-01-01 05:00:00,15470
...,...,...
39162,2022-06-20 19:00:00,17046
39163,2022-06-20 20:00:00,16900
39164,2022-06-20 21:00:00,16689
39165,2022-06-20 22:00:00,15697


## Import Holiday data

In [14]:
from datetime import date
import pandas as pd
import holidays

In [15]:
#create a datetime range
dates = pd.date_range(start='1/1/2018', end='20/6/2022')
dates

DatetimeIndex(['2018-01-01', '2018-01-02', '2018-01-03', '2018-01-04',
               '2018-01-05', '2018-01-06', '2018-01-07', '2018-01-08',
               '2018-01-09', '2018-01-10',
               ...
               '2022-06-11', '2022-06-12', '2022-06-13', '2022-06-14',
               '2022-06-15', '2022-06-16', '2022-06-17', '2022-06-18',
               '2022-06-19', '2022-06-20'],
              dtype='datetime64[ns]', length=1632, freq='D')

In [16]:
#produces categorical numerical values for the day of the week
#monday = 0
#sunday = 6
dates.weekday

Int64Index([0, 1, 2, 3, 4, 5, 6, 0, 1, 2,
            ...
            5, 6, 0, 1, 2, 3, 4, 5, 6, 0],
           dtype='int64', length=1632)

In [17]:
#create an object with all the holidays in Canada
canada_holidays = holidays.CountryHoliday('CA')

In [18]:
#investigate some known holiday dates
print(date(2019, 1, 1) in canada_holidays)
print(date(2018, 12, 25) in canada_holidays)
print(date(2018, 7, 1) in canada_holidays)

True
True
True


In [19]:
#can also get the name of the holiday
canada_holidays.get('2018-7-1')

'Canada Day'

In [20]:
def get_holidays(start='1/1/2018', stop='14/6/2022', country='CA', frequency='H'):
    """
    Takes in a start and stop date and a country.
    
    Produces a dataframe with a daily date time index and columns:
    day_of_week - numerical day of the week identifier 0 for monday
    holiday_bool - boolean true or false for holiday
    holiday_name - name of the holiday if holiday_bool is true
    
    Returns a dataframe
    """
    
    #generate the range of daily dates
    dates = pd.date_range(start=start, end=stop, freq=frequency)
    
    #create the holiday object
    country_holidays = holidays.CountryHoliday(country)

    #create a list for the holiday bool and name
    holiday_list = []
    
    #loop through the dates
    for date in dates:
        #true if holiday in object, false otherwise
        holiday_bool = date in country_holidays
        holiday_names = country_holidays.get(date)
        
        holiday_list.append([holiday_bool, holiday_names])
        
    #create return dataframe
    holidays_data = pd.DataFrame(holiday_list, index=dates, columns=['holiday_bool', 'holiday_name'])
                  
    return holidays_data

In [21]:
def get_days_dummies(start='1/1/2018', stop='31/12/2018', frequency='H'):
    """
    Takes in a start and stop date and frequency.
    
    Produces a dataframe with a date time index at the frequency input and columns:
    weekday_id - numerical day of the week identifier 0 for monday
    
    Returns a dataframe
    """
    
    #generate the range of daily dates
    dates = pd.date_range(start=start, end=stop, freq=frequency)
    
    #create a dataframe of weekday categories
    days = pd.DataFrame(list(dates.weekday), index=dates, columns=['weekday_id'])
    
    days = pd.get_dummies(days['weekday_id'])
    
    columns = ['mon', 'tue', 'wed', 'thur', 'fri', 'sat', 'sun']
    
    days.columns = columns
    
    return days
    

In [22]:
holiday_df = get_holidays(start='2018-01-01', stop='2022-06-21')
holiday_df['time'] = holiday_df.index
holiday_df.reset_index(drop=True, inplace=True)
# replace True/False with 1/0
holiday_df["holiday_bool"] = holiday_df["holiday_bool"].astype(int)
# drop name
holiday_df = holiday_df.drop(columns=['holiday_name'])

holiday_df

,holiday_bool,time
0,1,2018-01-01 00:00:00
1,1,2018-01-01 01:00:00
2,1,2018-01-01 02:00:00
3,1,2018-01-01 03:00:00
4,1,2018-01-01 04:00:00
...,...,...
39164,0,2022-06-20 20:00:00
39165,0,2022-06-20 21:00:00
39166,0,2022-06-20 22:00:00
39167,0,2022-06-20 23:00:00


In [24]:
holiday_df.isnull().sum()

holiday_bool    0
time            0
dtype: int64

## Import weather data

As for weather data, we are using the 5 cities selected based on the peak demands for select communities across the province of Ontario in 2018 based on the [IESO](https://www.ieso.ca/en/Power-Data/Demand-Overview/Historical-Demand) website.

The 5 cities are:
- Toronto
- Ottawa
- London
- Thunder Bay & Kenora
- Sudbury

![Historical Demand Information](https://www.ieso.ca/-/media/Images/IESO/Charts-and-Graphs/Peak-Demand-graphic-oeb-yearbook.ashx?h=587&w=700&la=en&hash=955A98527A869A884324590013A98F0A)

### Loading the Weather Data API

[Meteostat](https://dev.meteostat.net/) is an open platform which provides free access to historical weather and climate data.

Documentation of the API can be found [HERE](https://dev.meteostat.net/python/api/point/#parameters) 

Paied weather API: https://openweathermap.org/api


In [50]:
pip install meteostat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [51]:
# Import Meteostat library and dependencies

from datetime import datetime
from meteostat import Hourly
from meteostat import Stations

#### Gather Station Information in Ontario

In [52]:
stations = Stations()
stations = stations.region('CA','ON')
stations = stations.fetch()
stations = stations.reset_index(level=0)
stations.head(5)

,id,name,country,region,wmo,icao,latitude,longitude,elevation,timezone,hourly_start,hourly_end,daily_start,daily_end,monthly_start,monthly_end
0,0CNUO,Rawson Lake,CA,ON,<NA>,<NA>,49.65,-93.72,358.0,America/Rainy_River,NaT,NaT,2000-01-01,2022-04-30,2000-01-01,2022-01-01
1,0CO7B,Kingsville Moe,CA,ON,<NA>,<NA>,42.04,-82.67,200.0,America/Toronto,NaT,NaT,2000-01-01,2022-07-13,2000-01-01,2021-01-01
2,0FV1F,Tillsonburg Wwtp,CA,ON,<NA>,<NA>,42.86,-80.72,213.0,America/Toronto,NaT,NaT,2000-01-01,2022-07-14,2000-01-01,2021-01-01
3,0FV2W,Mountainview,CA,ON,<NA>,<NA>,44.05,-77.30,108.0,America/Toronto,NaT,NaT,2000-01-01,2018-08-21,2000-01-01,2018-01-01
4,10RIK,Brockville Pcc,CA,ON,<NA>,<NA>,44.60,-75.67,96.0,America/Toronto,NaT,NaT,2000-01-01,2022-07-13,2000-01-01,2022-01-01


According to Canadian government [website](https://climate.weather.gc.ca/historical_data/search_historic_data_e.html), the 5 station name for the 5 city listed above are:
- LONDON A
- OTTAWA CDA RCS
- SUDBURY CLIMATE
- THUNDER BAY
- TORONTO CITY CENTRE

Those stations are all located near the populated areas.

In [53]:
## Extract Station ID
london = stations.query('name.str.contains("London")', engine='python').id.values[0]
ottawa = stations.query('name.str.contains("Ottawa CDA")', engine='python').id.values[0]
sudbury = stations.query('name.str.contains("Sudbury")', engine='python').id.values[0]
thunderbay = stations.query('name.str.contains("Thunder Bay")', engine='python').id.values[1]
toronto = stations.query('name.str.contains("Toronto")', engine='python').id.values[2]

### Extract Hourly Weather Info

[**Data Dictionary**](https://dev.meteostat.net/python/hourly.html#data-structure)

|Column    | Description |Type |
| ----------- | ----------- | ----------- |
| time      | The datetime of the observation |Datetime64      |
| temp   | The air temperature in °C       |Float64      |
| dwpt   | The dew point in °C      |Float64      |
| rhum   | The relative humidity in percent (%)       |Float64      |
| prcp   | The one hour precipitation total in mm       |Float64      |
| snow   | The snow depth in mm       |Float64      |
| wdir   | The average wind direction in degrees (°)       |Float64      |
| wspd   | The average wind speed in km/h       |Float64      |
| wpgt   | The peak wind gust in km/h       |Float64      |
| pres   | The average sea-level air pressure in hPa       |Float64      |
| tsun   | The one hour sunshine total in minutes (m)       |Float64      |
| coco   | The weather [condition code](https://dev.meteostat.net/formats.html#meteorological-data-units)       |Float64      |

In [54]:
## Define Start and End Time
start = datetime(2018, 6, 20)
end = datetime(2022,6,20)

In [55]:
london_data = Hourly(london, start, end).fetch().reset_index(level=0)
london_data

,time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
0,2018-06-20 00:00:00,21.0,17.0,78.0,NaN,NaN,200.0,18.4,NaN,1014.3,NaN,NaN
1,2018-06-20 01:00:00,20.0,16.0,78.0,NaN,NaN,200.0,16.6,NaN,1014.8,NaN,NaN
2,2018-06-20 02:00:00,19.0,16.0,83.0,NaN,NaN,210.0,16.6,NaN,1014.8,NaN,NaN
3,2018-06-20 03:00:00,19.0,15.1,78.0,NaN,NaN,200.0,9.4,NaN,1014.2,NaN,NaN
4,2018-06-20 04:00:00,19.0,16.0,83.0,NaN,NaN,NaN,0.0,NaN,1014.1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
35022,2022-06-19 20:00:00,21.0,5.0,35.0,0.0,NaN,320.0,16.6,NaN,1023.3,NaN,3.0
35023,2022-06-19 21:00:00,21.0,4.1,33.0,0.0,NaN,320.0,22.3,NaN,1023.3,NaN,3.0
35024,2022-06-19 22:00:00,19.0,3.2,35.0,0.0,NaN,290.0,13.0,NaN,1022.9,NaN,3.0
35025,2022-06-19 23:00:00,18.0,1.1,32.0,0.3,NaN,310.0,11.2,NaN,1023.0,NaN,8.0


In [56]:
ottawa_data = Hourly(ottawa, start, end).fetch().reset_index(level=0)
ottawa_data

,time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
0,2018-06-20 00:00:00,22.8,7.4,37.0,NaN,NaN,320.0,11.2,NaN,1012.3,NaN,NaN
1,2018-06-20 01:00:00,20.7,8.7,46.0,NaN,NaN,300.0,7.6,NaN,1012.1,NaN,NaN
2,2018-06-20 02:00:00,17.5,9.1,58.0,NaN,NaN,260.0,3.6,NaN,1012.5,NaN,NaN
3,2018-06-20 03:00:00,16.0,10.5,70.0,NaN,NaN,250.0,5.4,NaN,1012.5,NaN,NaN
4,2018-06-20 04:00:00,16.8,9.7,63.0,NaN,NaN,250.0,9.4,NaN,1012.3,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
34810,2022-06-19 20:00:00,20.9,4.4,34.0,0.0,NaN,340.0,21.0,NaN,1016.4,NaN,1.0
34811,2022-06-19 21:00:00,21.5,4.1,32.0,0.0,NaN,350.0,23.0,NaN,1016.0,NaN,1.0
34812,2022-06-19 22:00:00,21.2,3.4,31.0,0.0,NaN,350.0,26.0,NaN,1016.1,NaN,1.0
34813,2022-06-19 23:00:00,20.8,2.1,29.0,0.0,NaN,350.0,21.0,NaN,1015.9,NaN,1.0


In [57]:
sudbury_data = Hourly(sudbury, start, end).fetch().reset_index(level=0)
sudbury_data

,time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
0,2018-06-20 00:00:00,23.6,6.8,34.0,NaN,NaN,360.0,1.8,NaN,1014.3,NaN,NaN
1,2018-06-20 01:00:00,21.0,9.9,49.0,NaN,NaN,160.0,7.6,NaN,1014.2,NaN,NaN
2,2018-06-20 02:00:00,17.0,11.1,68.0,NaN,NaN,230.0,11.2,NaN,1014.7,NaN,NaN
3,2018-06-20 03:00:00,16.7,10.8,68.0,NaN,NaN,260.0,7.6,NaN,1014.6,NaN,NaN
4,2018-06-20 04:00:00,16.0,11.0,72.0,NaN,NaN,240.0,13.0,NaN,1014.2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
34986,2022-06-19 20:00:00,19.8,-0.3,26.0,0.0,NaN,290.0,13.0,NaN,1021.3,NaN,3.0
34987,2022-06-19 21:00:00,18.6,0.7,30.0,0.0,NaN,270.0,15.0,NaN,1020.8,NaN,3.0
34988,2022-06-19 22:00:00,17.5,1.9,35.0,0.0,NaN,270.0,8.0,NaN,1020.6,NaN,3.0
34989,2022-06-19 23:00:00,16.2,4.9,47.0,0.0,NaN,270.0,10.0,NaN,1020.6,NaN,3.0


In [58]:
thunderbay_data = Hourly(thunderbay, start, end).fetch().reset_index(level=0)
thunderbay_data

,time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
0,2018-06-20 00:00:00,23.0,11.1,47.0,NaN,NaN,180.0,13.0,NaN,1015.7,NaN,2.0
1,2018-06-20 01:00:00,21.0,11.1,53.0,NaN,NaN,220.0,7.6,NaN,1015.3,NaN,NaN
2,2018-06-20 02:00:00,19.0,11.1,60.0,NaN,NaN,NaN,0.0,NaN,1015.2,NaN,NaN
3,2018-06-20 03:00:00,16.0,12.0,77.0,NaN,NaN,219.0,0.0,NaN,1015.2,NaN,1.0
4,2018-06-20 04:00:00,14.0,12.0,88.0,NaN,NaN,NaN,0.0,NaN,1015.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
35056,2022-06-19 20:00:00,12.0,9.0,82.0,0.0,NaN,90.0,22.3,NaN,1016.6,NaN,1.0
35057,2022-06-19 21:00:00,13.0,10.0,82.0,0.0,NaN,50.0,7.6,NaN,1015.0,NaN,2.0
35058,2022-06-19 22:00:00,14.0,10.0,77.0,0.0,NaN,90.0,14.8,NaN,1014.3,NaN,3.0
35059,2022-06-19 23:00:00,14.0,10.0,77.0,0.0,NaN,80.0,16.6,NaN,1014.1,NaN,3.0


In [59]:
toronto_data = Hourly(toronto, start, end).fetch().reset_index(level=0)
toronto_data

,time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
0,2018-06-20 00:00:00,20.0,8.3,47.0,NaN,NaN,NaN,0.0,NaN,1014.4,NaN,NaN
1,2018-06-20 01:00:00,19.4,7.5,46.0,NaN,NaN,NaN,NaN,NaN,1014.2,NaN,NaN
2,2018-06-20 02:00:00,19.7,7.1,44.0,NaN,NaN,NaN,NaN,NaN,1014.5,NaN,NaN
3,2018-06-20 03:00:00,18.8,9.0,53.0,NaN,NaN,NaN,0.0,NaN,1014.5,NaN,NaN
4,2018-06-20 04:00:00,17.9,7.6,51.0,NaN,NaN,NaN,NaN,NaN,1014.1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
34862,2022-06-19 20:00:00,21.0,-3.5,19.0,0.0,NaN,331.0,21.6,NaN,1020.9,NaN,3.0
34863,2022-06-19 21:00:00,20.4,-4.7,18.0,0.0,NaN,334.0,22.3,NaN,1020.7,NaN,3.0
34864,2022-06-19 22:00:00,19.9,-5.9,17.0,0.0,NaN,336.0,21.2,NaN,1020.8,NaN,3.0
34865,2022-06-19 23:00:00,18.8,-7.6,16.0,0.0,NaN,333.0,19.4,NaN,1020.8,NaN,3.0


### Processing Missing Values in Weather Data

#### Summary of Missing Data in Each City

In [60]:
london_data.isnull().sum()

time        0
temp        0
dwpt        0
rhum        0
prcp    22229
snow    35027
wdir      766
wspd        9
wpgt    35027
pres      156
tsun    35027
coco    28350
dtype: int64

In [61]:
ottawa_data.isnull().sum()

time        0
temp        0
dwpt       55
rhum       55
prcp    11968
snow    34815
wdir        1
wspd        0
wpgt    34815
pres       51
tsun    34815
coco    34642
dtype: int64

In [62]:
sudbury_data.isnull().sum()

time        0
temp        2
dwpt        2
rhum        2
prcp    22193
snow    34991
wdir      237
wspd        3
wpgt    34991
pres      163
tsun    34991
coco    27472
dtype: int64

In [63]:
thunderbay_data.isnull().sum()

time        0
temp        0
dwpt        0
rhum        0
prcp    34436
snow    35061
wdir       57
wspd        0
wpgt    35061
pres       10
tsun    35061
coco      677
dtype: int64

In [64]:
toronto_data.isnull().sum()

time        0
temp        8
dwpt        8
rhum        8
prcp    12190
snow    34867
wdir    19168
wspd    14777
wpgt    34867
pres      171
tsun    34867
coco    34696
dtype: int64

#### Processing Steps

**Strategy of Processing Missing Values**

If a variable is missing most of it's value (more than 70%), we choose to drop the variables.

For variables such as wind speed and precipitation, if the data point is empty we will fill the missing point with 0.

For other variable such as wind direction, we will fill the missing value based on the value before or after the missing data point. Because those weather factors usually does not change much with in one hour.



In [65]:
## Dropping useless columns from data set
useless_vars = ['snow','wpgt','tsun','wdir','coco']
london_data = london_data.drop(columns = useless_vars)
ottawa_data = ottawa_data.drop(columns = useless_vars)
sudbury_data = sudbury_data.drop(columns = useless_vars)
thunderbay_data = thunderbay_data.drop(columns = useless_vars)
toronto_data = toronto_data.drop(columns = useless_vars)

In [66]:
## fill missing values of wind speed and precipitation with zero
vars1 = ['prcp','wspd']
london_data.update(london_data[vars1].fillna(0))
ottawa_data.update(london_data[vars1].fillna(0))
sudbury_data.update(london_data[vars1].fillna(0))
thunderbay_data.update(london_data[vars1].fillna(0))
toronto_data.update(london_data[vars1].fillna(0))

In [67]:
## Fill other variable's missing value with the value before the missing datapoint
vars2 = london_data.columns.tolist()

london_data[vars2]=london_data[vars2].ffill()
ottawa_data[vars2]=ottawa_data[vars2].ffill()
sudbury_data[vars2]=sudbury_data[vars2].ffill()
thunderbay_data[vars2]=thunderbay_data[vars2].ffill()
toronto_data[vars2]=toronto_data[vars2].ffill()

In [68]:
london_data.isnull().sum()

time    0
temp    0
dwpt    0
rhum    0
prcp    0
wspd    0
pres    0
dtype: int64

In [69]:
ottawa_data.isnull().sum()

time    0
temp    0
dwpt    0
rhum    0
prcp    0
wspd    0
pres    0
dtype: int64

In [70]:
sudbury_data.isnull().sum()

time    0
temp    0
dwpt    0
rhum    0
prcp    0
wspd    0
pres    0
dtype: int64

In [71]:
thunderbay_data.isnull().sum()

time    0
temp    0
dwpt    0
rhum    0
prcp    0
wspd    0
pres    0
dtype: int64

In [72]:
toronto_data.isnull().sum()

time    0
temp    0
dwpt    0
rhum    0
prcp    0
wspd    0
pres    0
dtype: int64

By now, all missing values have been filled.

### Combine All the Weather Data

In [73]:
## Renaming columns of each city's weather data
london_data.columns = [str(col) + '_london' for col in london_data.columns]
ottawa_data.columns = [str(col) + '_ottawa' for col in ottawa_data.columns]
sudbury_data.columns = [str(col) + '_sudbury' for col in sudbury_data.columns]
thunderbay_data.columns = [str(col) + '_thunderbay' for col in thunderbay_data.columns]
toronto_data.columns = [str(col) + '_toronto' for col in toronto_data.columns]

In [74]:
## Combining all the weather data
weather = pd.merge(london_data, ottawa_data, how='inner', left_on='time_london',right_on='time_ottawa')
weather = pd.merge(weather, sudbury_data, how='inner', left_on='time_london',right_on='time_sudbury')
weather = pd.merge(weather, thunderbay_data, how='inner', left_on='time_london',right_on='time_thunderbay')
weather = pd.merge(weather, toronto_data, how='inner', left_on='time_london',right_on='time_toronto')
## drop the other time columns, only keep the first one
weather = weather.drop(columns = ['time_ottawa','time_sudbury','time_thunderbay','time_toronto'])
## change time_london to time
weather = weather.rename(columns={'time_london': 'time'})
weather

,time,temp_london,dwpt_london,rhum_london,prcp_london,wspd_london,pres_london,temp_ottawa,dwpt_ottawa,rhum_ottawa,...,rhum_thunderbay,prcp_thunderbay,wspd_thunderbay,pres_thunderbay,temp_toronto,dwpt_toronto,rhum_toronto,prcp_toronto,wspd_toronto,pres_toronto
0,2018-06-20 00:00:00,21.0,17.0,78.0,0.0,18.4,1014.3,22.8,7.4,37.0,...,47.0,0.0,18.4,1015.7,20.0,8.3,47.0,0.0,18.4,1014.4
1,2018-06-20 01:00:00,20.0,16.0,78.0,0.0,16.6,1014.8,20.7,8.7,46.0,...,53.0,0.0,16.6,1015.3,19.4,7.5,46.0,0.0,16.6,1014.2
2,2018-06-20 02:00:00,19.0,16.0,83.0,0.0,16.6,1014.8,17.5,9.1,58.0,...,60.0,0.0,16.6,1015.2,19.7,7.1,44.0,0.0,16.6,1014.5
3,2018-06-20 03:00:00,19.0,15.1,78.0,0.0,9.4,1014.2,16.0,10.5,70.0,...,77.0,0.0,9.4,1015.2,18.8,9.0,53.0,0.0,9.4,1014.5
4,2018-06-20 04:00:00,19.0,16.0,83.0,0.0,0.0,1014.1,16.8,9.7,63.0,...,88.0,0.0,0.0,1015.0,17.9,7.6,51.0,0.0,0.0,1014.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34589,2022-06-19 20:00:00,21.0,5.0,35.0,0.0,16.6,1023.3,20.9,4.4,34.0,...,82.0,0.0,22.3,1016.6,21.0,-3.5,19.0,0.0,9.4,1020.9
34590,2022-06-19 21:00:00,21.0,4.1,33.0,0.0,22.3,1023.3,21.5,4.1,32.0,...,82.0,0.0,7.6,1015.0,20.4,-4.7,18.0,0.0,9.4,1020.7
34591,2022-06-19 22:00:00,19.0,3.2,35.0,0.0,13.0,1022.9,21.2,3.4,31.0,...,77.0,0.0,14.8,1014.3,19.9,-5.9,17.0,0.0,3.6,1020.8
34592,2022-06-19 23:00:00,18.0,1.1,32.0,0.3,11.2,1023.0,20.8,2.1,29.0,...,77.0,0.0,16.6,1014.1,18.8,-7.6,16.0,0.0,7.6,1020.8


## Combining price, supply, demand, holiday, weather together

In [75]:
historical_data = pd.merge(price, weather, how='inner', on = 'time')

In [76]:
historical_data = pd.merge(historical_data, holiday_df, how='inner', on = 'time')

In [77]:
historical_data = pd.merge(historical_data, supply , how='inner', on = 'time')

In [78]:
historical_data = pd.merge(historical_data, demand , how='inner', on = 'time')

In [79]:
historical_data

,time,HOEP,temp_london,dwpt_london,rhum_london,prcp_london,wspd_london,pres_london,temp_ottawa,dwpt_ottawa,...,pres_toronto,holiday_bool,NUCLEAR,GAS,HYDRO,WIND,SOLAR,BIOFUEL,Total,Ontario Demand
0,2018-06-20 00:00:00,22.73,21.0,17.0,78.0,0.0,18.4,1014.3,22.8,7.4,...,1014.4,0,10201,448,4120,840,0,16,15625,14062
1,2018-06-20 01:00:00,13.35,20.0,16.0,78.0,0.0,16.6,1014.8,20.7,8.7,...,1014.2,0,10211,323,3706,710,0,13,14963,13307
2,2018-06-20 02:00:00,8.99,19.0,16.0,83.0,0.0,16.6,1014.8,17.5,9.1,...,1014.5,0,10217,329,3693,663,0,14,14916,12925
3,2018-06-20 03:00:00,12.68,19.0,15.1,78.0,0.0,9.4,1014.2,16.0,10.5,...,1014.5,0,10214,340,3727,742,0,12,15035,12630
4,2018-06-20 04:00:00,13.29,19.0,16.0,83.0,0.0,0.0,1014.1,16.8,9.7,...,1014.1,0,10216,449,3761,750,0,12,15188,12476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34589,2022-06-19 20:00:00,63.87,21.0,5.0,35.0,0.0,16.6,1023.3,20.9,4.4,...,1020.9,0,9717,135,4801,2234,7,0,16894,14811
34590,2022-06-19 21:00:00,65.32,21.0,4.1,33.0,0.0,22.3,1023.3,21.5,4.1,...,1020.7,0,9711,136,4956,2118,1,0,16922,14973
34591,2022-06-19 22:00:00,415.54,19.0,3.2,35.0,0.0,13.0,1022.9,21.2,3.4,...,1020.8,0,9716,138,4322,1999,0,0,16175,14204
34592,2022-06-19 23:00:00,34.19,18.0,1.1,32.0,0.3,11.2,1023.0,20.8,2.1,...,1020.8,0,9726,138,3864,1819,0,0,15547,12996


In [ ]:
historical_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34594 entries, 0 to 34593
Data columns (total 41 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   time             34594 non-null  datetime64[ns]
 1   HOEP             34594 non-null  float64       
 2   temp_london      34594 non-null  float64       
 3   dwpt_london      34594 non-null  float64       
 4   rhum_london      34594 non-null  float64       
 5   prcp_london      34594 non-null  float64       
 6   wspd_london      34594 non-null  float64       
 7   pres_london      34594 non-null  float64       
 8   temp_ottawa      34594 non-null  float64       
 9   dwpt_ottawa      34594 non-null  float64       
 10  rhum_ottawa      34594 non-null  float64       
 11  prcp_ottawa      34594 non-null  float64       
 12  wspd_ottawa      34594 non-null  float64       
 13  pres_ottawa      34594 non-null  float64       
 14  temp_sudbury     34594 non-null  float

## Export final CSV

In [80]:
from google.colab import files
historical_data.to_csv('historical_data.csv',index=False) 
files.download('historical_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>